In [2]:
# import all of the necessary libraries 
from __future__ import division
import pandas as pd


In [ ]:

# Get TPM using tpm function 
def counts_to_tpm(counts_matrix):
    data_cols = counts_matrix.columns[1:]
    rpk = counts_matrix[data_cols].divide((counts_matrix['Length']/1000), axis=0)
    sums_per_million = rpk.sum(axis=0)/(10**6)
    tpm = rpk.divide(sums_per_million)
    
    return tpm

In [3]:
# get the list of protein coding genes 
pcgs = pd.read_table('gtf_protein_simpified.csv', names=['Geneid'])
pcgs.head(1) # perfect

/home/wschrein/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


,Geneid
0,WBGene00010957


In [4]:
# current gene IDs 
gene_names = pd.read_csv('c_elegans.PRJNA13758.current.geneIDs.txt', 
      names=['numba', 'Geneid', 'common_name', 'transcript', 'status']) 

gene_names.drop(['numba', 'status'],axis=1,inplace=True)

In [5]:
# these are the genes that are up in heat shock; dogs and repeats have not been filtered out

up = pd.read_csv('/home/wschrein/final/Figure1/up_genes_no_intron_filtering.csv')

In [16]:
# This file was genereated by using feature counts against BAM files
# the SAF file generated in A. Make intergenic files

negative_dogs = pd.read_table('negative_iDOGS.txt', comment='#', index_col=0)

negative_dogs.head(4)

/home/wschrein/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.


,Chr,Start,End,Strand,Length,/home/wschrein/scratch/nuevo/att_1/bams/Sample1Aligned.out.sam.sorted.bam,/home/wschrein/scratch/nuevo/att_1/bams/Sample2Aligned.out.sam.sorted.bam,/home/wschrein/scratch/nuevo/att_1/bams/Sample3Aligned.out.sam.sorted.bam,/home/wschrein/scratch/nuevo/att_1/bams/Sample4Aligned.out.sam.sorted.bam,/home/wschrein/scratch/nuevo/att_1/bams/Sample5Aligned.out.sam.sorted.bam,/home/wschrein/scratch/nuevo/att_1/bams/Sample6Aligned.out.sam.sorted.bam
Geneid,,,,,,,,,,,
WBGene00023193,I,3900,3920,-,21,0,0,0,0,0,0
WBGene00022277,I,10223,10243,-,21,0,4,0,8,0,0
WBGene00022278,I,26772,26792,-,21,10,5,6,0,3,1
WBGene00022279,I,32473,32493,-,21,66,103,90,61,16,56


In [17]:
! wc -l negative_iDOGS.txt  # how many lines

15525 negative_iDOGS.txt


In [18]:
negative_dogs = negative_dogs[negative_dogs['Strand']=="-"]

In [19]:
# change the column names to something more palatable 

negative_dogs = negative_dogs.rename(columns={'/home/wschrein/scratch/nuevo/att_1/bams/Sample1Aligned.out.sam.sorted.bam':'ctr_1',
                          '/home/wschrein/scratch/nuevo/att_1/bams/Sample2Aligned.out.sam.sorted.bam':'ctr_3',
                          '/home/wschrein/scratch/nuevo/att_1/bams/Sample3Aligned.out.sam.sorted.bam':'ctr_4',
                          '/home/wschrein/scratch/nuevo/att_1/bams/Sample4Aligned.out.sam.sorted.bam':'hs_1',
                          '/home/wschrein/scratch/nuevo/att_1/bams/Sample5Aligned.out.sam.sorted.bam':'hs_3',
                          '/home/wschrein/scratch/nuevo/att_1/bams/Sample6Aligned.out.sam.sorted.bam':'hs_4'
                          })
negative_dogs.head(1)

,Chr,Start,End,Strand,Length,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4
Geneid,,,,,,,,,,,
WBGene00023193,I,3900,3920,-,21,0,0,0,0,0,0


In [20]:
# make an info column of things that you might need later 
info_nd = negative_dogs.reset_index()
info_nd = info_nd[['Geneid', 'Chr', 'Start', 'End']]

In [21]:
# now get rid of them 
negative_dogs.drop(['Chr', 'Start', 'End'], inplace=True, axis=1)

In [22]:
negative_dogs_tpm = counts_to_tpm(negative_dogs)
negative_dogs_tpm.drop('Length', axis=1, inplace=True)

In [23]:
negative_dogs_tpm.head(1)

,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4
Geneid,,,,,,
WBGene00023193,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
data_cols = negative_dogs_tpm.columns[0:]
data_cols

Index([u'ctr_1', u'ctr_3', u'ctr_4', u'hs_1', u'hs_3', u'hs_4'], dtype='object')

In [25]:
negative_dogs_tpm_filterd = negative_dogs_tpm.loc[negative_dogs_tpm[data_cols].sum(axis=1) > 0]

In [26]:
negative_dogs_tpm_filterd.head(1)

,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4
Geneid,,,,,,
WBGene00022277,0.0,158.868854,0.0,128.392367,0.0,0.0


In [27]:
negative_dogs_tpm_filterd = negative_dogs_tpm_filterd.copy()  # make copy to avoid slicing error message

In [28]:
negative_dogs_tpm_filterd.head()

,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4
Geneid,,,,,,
WBGene00022277,0.000000,158.868854,0.000000,128.392367,0.000000,0.000000
WBGene00022278,343.453771,198.586067,235.793445,0.000000,81.741642,17.404926
WBGene00022279,2266.794889,4090.872984,3536.901674,978.991799,435.955424,974.675833
WBGene00021679,68.690754,0.000000,550.184705,0.000000,0.000000,34.809851
WBGene00189949,0.000000,0.000000,0.000000,0.000000,0.000000,34.809851


# Calculate the intergenic Ratio

In [29]:
negative_dogs_tpm_filterd['ctrl_total'] = negative_dogs_tpm_filterd.ctr_1 + negative_dogs_tpm_filterd.ctr_3 + negative_dogs_tpm_filterd.ctr_4

In [30]:
negative_dogs_tpm_filterd['hs_total'] = negative_dogs_tpm_filterd.hs_1 + negative_dogs_tpm_filterd.hs_3 + negative_dogs_tpm_filterd.hs_4

In [31]:
# add one to all control to avoid divide by 0 error

negative_dogs_tpm_filterd['ctrl_total'] = negative_dogs_tpm_filterd.ctrl_total +1

In [34]:
# Calculate the intergenic HS ratio
negative_dogs_tpm_filterd['ratio'] = negative_dogs_tpm_filterd['hs_total'] / negative_dogs_tpm_filterd['ctrl_total']

In [ ]:
negative_dogs_tpm_filterd.head()

## Make it so that only two replicates need to be non-zero ###

In [36]:
# Make it so that only two replicates have to be non-zero
_1_3 = negative_dogs_tpm_filterd[(negative_dogs_tpm_filterd['hs_1']!=0) & (negative_dogs_tpm_filterd['hs_3']!=0)]
_1_4 = negative_dogs_tpm_filterd[(negative_dogs_tpm_filterd['hs_1']!=0) & (negative_dogs_tpm_filterd['hs_4']!=0)]
_3_4 = negative_dogs_tpm_filterd[(negative_dogs_tpm_filterd['hs_3']!=0) & (negative_dogs_tpm_filterd['hs_4']!=0)]


In [51]:
df_concat = pd.concat([_1_3,_1_4,_3_4]) # now add them on top of each other 
df_concat.reset_index(inplace=True)
df_concat.drop_duplicates(inplace=True)  # remove duplicates 

len(df_concat) # how many rows

2649

In [52]:
# rename the file to neg_filtered
neg_filtered = df_concat

In [54]:
# get list of Genes with an intergenic ratio of greater than 2
# By Trial an error this seems to be a good ratio to filter out DoGs

neg_filtered = neg_filtered[neg_filtered['ratio']>2]
neg_filtered.reset_index(inplace=True, drop=True)
neg_filtered.head(1)

,Geneid,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio
0,WBGene00004274,68.690754,0.0,157.19563,256.784734,190.730498,156.64433,226.886384,604.159562,2.662829


In [55]:
# we are most concerned with filtering out pcgs
# so merge with list of protein coding genes
# pcgs were determined by parsing gtf file for genes 
# annotated as pcgs

neg_filtered = pd.merge(neg_filtered, pcgs, how='inner', on='Geneid')


In [58]:
# import intron retention scores to further filter out
# this file is a shortened version of the Intron Retention Generated in 
# Figure1/Calculate_Intron_Retention_Score
# it has been mereged with HS up genes, so it only has scores for HS up genes

s2 = pd.read_csv('s2_scores.csv') 
s2.drop('Unnamed: 0', inplace=True, axis=1)  # get rid of unnecessary column

s2.rename(columns={'WB_ID': 'Geneid', 'ratio':'IR_score'}, inplace = True)  # rename column for merging

In [59]:
neg_filtered.head(1)

,Geneid,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio
0,WBGene00004274,68.690754,0.0,157.19563,256.784734,190.730498,156.64433,226.886384,604.159562,2.662829


In [60]:
neg_filtered = pd.merge(neg_filtered, s2, on='Geneid')


In [61]:
neg_filtered.head(1)

,Geneid,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio,IR_score
0,WBGene00000788,0.0,0.0,0.0,128.392367,54.494428,0.0,1.0,182.886795,182.886795,0.016073


In [62]:
# Create list of genes with IJ ratio greater than2 and IR score greater than 0.4
# by examination with genome browser this seems to filter out most bad DoGs


neg_filtered = neg_filtered[neg_filtered.IR_score>.4]

In [63]:
neg_filtered = neg_filtered[['Geneid']]

In [65]:
## This file is used in Figure1/D1. Filter_the_up_Genes.ipynb
## to filter out interfering DoG on the - strand

neg_filtered.to_csv('up_negative_bad_dogs.csv',index=None)